<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
# импотрируем нужные нам биб-ки

import pandas as pd
import numpy as np

from sklearn.metrics import r2_score

In [2]:
# загрузим наши данные и выведим первые 5 строк
try:
    data = pd.read_csv('/Users/alex/Downloads/insurance.csv')
except:
    data = pd.read_csv('/datasets/insurance.csv')
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [4]:
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [5]:
# изменим ти данных для столбоц Возраст и Зарплата

data = data.astype({'Возраст':'int64', 
                    'Зарплата':'int64'})

print(f'Тип данных столбца Возраст: {data["Возраст"].dtype}')
print(f'Тип данных столбца Зарплата: {data["Зарплата"].dtype}')

Тип данных столбца Возраст: int64
Тип данных столбца Зарплата: int64


In [6]:
# проверим дф на дубликаты

data.duplicated().sum()

153

In [7]:
# избавимся от дубликатов

data.drop_duplicates(inplace = True)
data.duplicated().sum()

0

In [8]:
# взглянем нет ли аномальных 2.5 человека)

data['Члены семьи'].unique()

array([1, 0, 2, 4, 3, 5, 6])

In [9]:
# посмотрим имеются ли пропуски

data.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

In [10]:
# взгялем, нет ли мультиколлениарности 

data.corr()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
Пол,1.000000,0.001953,0.015456,-0.007315,0.011565
Возраст,0.001953,1.000000,-0.017386,-0.009064,0.654964
Зарплата,0.015456,-0.017386,1.000000,-0.031687,-0.013123
Члены семьи,-0.007315,-0.009064,-0.031687,1.000000,-0.039303
Страховые выплаты,0.011565,0.654964,-0.013123,-0.039303,1.000000


Посмотрели общую информацию о дфе, избавились от дубликатов, пропусков нет, мультиколлениарность не обнаружена.

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Качество линейной регрессии не изменится

**Обоснование:** 


Запишем преобразование признаков так:
$$
U = XP
$$

где, $U$ - преобразованные признаки, $P$ - обратимая матрица.


Тогда предасказания будет: $A = UW$

Формула обучения будет иметь вид: $W = (U^T U)^{-1} U^T y$ , где W - вектор весов

Подставим наше преобразование в __формулу обучения__ =>   
$$
W = ((XP)^T (XP))^{-1} (XP)^T y
$$

__Вспоминая свойства матриц, а именно:__ 
- По определению, квадратная матрица $A$ называется обратимой, если существует матрица $B$, удовлетворяющая условиям $$AB = BA = E$$
- Если по определению $A$ обратима, то обратная к ней обозначается, как $A^{-1}$ и условие выше можно зависать так: $$AA^{-1} = A^{-1}A = E$$
- Так же нам понадобятся следующие свойства матриц: $$(AB)^{T} = B^{T}A^{T}$$ $$(AB)^{-1} = B^{-1}A^{-1}$$ $$AE = EA = A$$

Аккуратно преобразуем правую часть, расскрыв скобки:
$$W = ((XP)^T XP)^{-1} (XP)^T y =>$$
$$W = (P^{T}X^{T}XP)^{-1} P^{T}X^{T} y =>$$
$$W = P^{-1}(P^{T})^{-1}(X^{T}X)^{-1}P^{T}X^{T}y =>$$

При умножении матрицы на обратную ей получается единичная матрица $E$, следовательно => $(P^{T})^{-1}P^{T} = E$, тогда
$$W = (X^{T}X)^{-1}EP^{-1}X^{T}y$$

Т.к. при умножении на единичную матрицу мы получим ту же самую матрицу - избавимся от $E$
$$W = P^{-1}(X^{T}X)^{-1}X^{T}y =>$$
Из описания мы знаем, что $w = (X^T X)^{-1} X^T y$ преобразуем предыдущую формулу:
$$W = P^{-1}w$$
Далее подставим формулу вектора в формулу предсказаний:
$$A = UW =>$$
$$A = XPP^{-1}w =>$$
Сократим $P$ и $P^{-1}$ тк получим единичную матрицу:
$$A = XEw =>$$
Избавимся от $E$ получим: 
$$A = Xw$$

__Вывод__

Выведенная формула, где признаки умножили на обратимую матрицу - идентична данной вначале. Следовательно качесто линейной регрессии не будет меняться

## Алгоритм преобразования
Предложите алгоритм преобразования данных для решения задачи.

**Алгоритм**

Чтобы зашифровать данные, в качестве алоритма преобразования мы проделаем следующее:
- сгенерируем обратимую матрицу(квадратную), размерность которой будет соответсвовать кол-ву столбцов исходному дфу , т.е 4х4
- умножим исходную матрицу признаков на созданную
- вычислить качество модели на полученной матрице и сравнить с исходной

**Обоснование**

Как мы выяснили в пукте №2, предсказания с домножением и исходные не отличаются. Поэтому R2 на измененной и исходной должны получиться одинаковыми. Таким оброзом можно считать, что наше шифрование будет удачным.

## Проверка алгоритма

In [11]:
# выделим наши фичи и таргет

features = data.drop('Страховые выплаты', axis = 1)
target = data['Страховые выплаты']

In [12]:
# создадим рандомную матрицу

matrix_random = np.random.randn(4, 4)
matrix_random

array([[ 0.82313244,  0.93688419,  0.56261894, -0.46382128],
       [-0.24967627,  1.7979529 , -1.87444369,  0.59791352],
       [ 0.5207969 , -2.67505948,  1.27761508, -1.69521318],
       [ 1.44792643, -0.43968532,  0.6884294 , -2.41520751]])

In [13]:
# проверим нащу рандомную матрицу на обратимость

try:
    matrix_inverse = np.linalg.inv(matrix_random)
    print('Обратная матрица - существует')
except:
    print('Обратная матрица - не существует')

Обратная матрица - существует


Создадим матрицу из исxодных признаков, далее умножим на созданную обратную матрицу(рандомную), и создадим новый дф и взлянем на наши зашифрованные признаки- features_encrypted

In [14]:
# исходная матрица
matrix = features.values

# зашифрованная матрица(умноженная на созданную обратную(рандомную) матрицу)
matrix_encrypted = matrix @ matrix_inverse

# создадим зашифрованные признаки в виде дф
features_encrypted = pd.DataFrame(matrix_encrypted, columns = features.columns)
features_encrypted.head(5)

,Пол,Возраст,Зарплата,Члены семьи
0,-64935.803305,-87799.471874,-90711.302696,54403.653053
1,-49777.974757,-67289.644823,-69532.200025,41704.738250
2,-27516.980845,-37192.839396,-38435.348862,23054.321104
3,-54571.120633,-73796.513070,-76234.623845,45718.329493
4,-34179.845537,-46209.524538,-47746.369027,28637.006841


__Создадим класс Линейной Регрессии__

(хотя в принципе можно импортировать, и по идее должно быть тоже самое)

In [19]:
# создадим класс лр

class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis = 1)
        y = train_target
        w = np.linalg.inv(X.T@ X) @ X.T @ y
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

In [20]:
# запишем наш класс лр в переменную model
model = LinearRegression()

# обучим на первичных признаках и запредиктим
model.fit(features, target)
predictions = model.predict(features)

# расчитаем метрику R2
r2 = r2_score(target, predictions)

print(f'Метрика R2 для признаков в исходном состоянии равна: {r2}')

Метрика R2 для признаков в исходном состоянии равна: 0.4302010046633359


далее будем использовать наши измененные/зашифрованные признаки

In [21]:
# инициализируем нашу модель
model_encrypted = LinearRegression()

# обучим на зашифрованных признаках и запредиктим
model_encrypted.fit(features_encrypted, target)
predictions_encrypted = model_encrypted.predict(features_encrypted)

# расчитаем метрику R2
r2_encrypted = r2_score(target, predictions_encrypted)

print(f'Метрика R2 для признаков в исходном состоянии равна: {r2_encrypted}')

Метрика R2 для признаков в исходном состоянии равна: 0.4302010046629


__ВЫВОД__

Как мы видим из выше представленных расчетов - метрика R2 практически не отличается, следовательно можно сказать, что умножение исходных данных на случайную обратимую матрицу - может быть использовано для их шифрования, при этом потери качетства не будет.